### Test

First of all I suggest you to produce a single compilable macro that contains the likelihood you want to use, you can find in other tutorial how to define the likelihood [here](https://xenon1t.github.io/Xephyr/docs/tutorials.html). The idea of this macro is to define a function that you can use to retrieve the likelihood, so that you have it defined in one place only. You can get inspiration on how to write this function from [here](https://github.com/XENON1T/SR1Results/blob/master/StatisticalAnalyses/xephyr_sr1_likelihood/src/likelihoodDef.cxx)

In [1]:
.x loadXephyr.C

Info in <TUnixSystem::ACLiC>: creating shared library /home/pan/work/XEPHYR_PKG/Xephyr/notebooks/./../src/dataHandler_cxx.so
Info in <TUnixSystem::ACLiC>: creating shared library /home/pan/work/XEPHYR_PKG/Xephyr/notebooks/./../src/XePdfObjects_cxx.so
Info in <TUnixSystem::ACLiC>: creating shared library /home/pan/work/XEPHYR_PKG/Xephyr/notebooks/./../src/XeLikelihoods_cxx.so
Info in <TUnixSystem::ACLiC>: creating shared library /home/pan/work/XEPHYR_PKG/Xephyr/notebooks/./../src/AsymptoticExclusion_cxx.so
Info in <TUnixSystem::ACLiC>: creating shared library /home/pan/work/XEPHYR_PKG/Xephyr/notebooks/./../src/ToyGenerator_cxx.so
Info in <TUnixSystem::ACLiC>: creating shared library /home/pan/work/XEPHYR_PKG/Xephyr/notebooks/./../src/ToyFitterExclusion_cxx.so


(int) 0


In [2]:
// add path to include files to ROOT (in the file I'm going to load there are some Xephyr includes) 
gInterpreter->AddIncludePath("../src");  // in this case is just XEPHYR src from next dir.

// now load the function to retrieve the likelihood

In [3]:
.L ../../SR1/StatisticalAnalyses/xephyr_sr1_likelihood/src/likelihoodDef.cxx

In [4]:
pdfLikelihood likeHood = getTheLikelihood(50.);  // 50. GeV mass

pdfLikelihood - INFO: bkg component named hband added to    SAFEGUARDED
pdfLikelihood::initialize - INFO :  initialize..... 
	Likelihood::addParameter - Info : Adding parameter Sigma  with ID -1  to PL xe1T
INFO :  adding sys for BKG component hband
	Likelihood::addParameter - Info : Adding parameter ERscale  with ID 1  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _PY  with ID 2  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _RF  with ID 3  to PL xe1T
INFO :  adding sys for SIGNAL component wimp_0050gev
	Likelihood::addParameter - Info : Adding parameter _gamma_  with ID 4  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _alpha_  with ID 5  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _eta_  with ID 6  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _acceptance_par_  with ID 7  to PL xe1T

------ WARNING -------  Safeguard is turned OFF altough you have set components to be safeguarded this is ignored --

Error in <TNtuple::SetBranchAddress>: The pointer type given "Double_t" (8) does not correspond to the type needed "Float_t" (5) by the branch: cs1
Error in <TNtuple::SetBranchAddress>: The pointer type given "Double_t" (8) does not correspond to the type needed "Float_t" (5) by the branch: cs2


In [5]:
ToyFitterExclusion fitter("treeName.root");

In [6]:
fitter.setTheLikelihood(&likeHood);

In [7]:
double qstat = fitter.computeTS(0.1);

Finding maximum of xe1T
Total of 6 active parameters, don't freeze parameters of interest, Minuit fits 6 param.
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest        0                 
  1 ERscale              Free               0                 
  2 _PY                  Nuisance           0                 
  3 _RF                  Nuisance           0                 
  4 _gamma_              Fixed              0                 
  5 _alpha_              Nuisance           0                 
  6 _eta_                Fixed              0                 
  7 _acceptance_par_     Nuisance           0                 
Stat::pdfLikelihood::computeTheLogLikelihood - WARNING : NsFs + NbFb <= 0 
Stat::pdfLikelihood::computeTheLogLikelihood - WARNING : NsFs + NbFb <= 0 
Stat::pdfLikelihood::computeTheLogLikelihood - WARNING : NsFs + NbFb <= 0 
Stat::pdfLikelihood::computeTheLogLikelihood - WARNING : NsFs + NbFb <= 0 
Stat::

In [8]:
fitter.setPrintLevel(0);

In [9]:
qstat

(double) 684.722417


In [10]:
fitter.fit(10,"treeName");

ToyExclusion::fit - DEBUG : Matched tree name, treeName0
ToyExclusion::measureParameters - INFO : Randomizing parameters:
ToyExclusion::----> - INFO : Skipping paramater: Sigma
ToyExclusion::----> - INFO : Skipping paramater: ERscale
ToyExclusion::----> - INFO : new T0-Value: _PY = -0.88
ToyExclusion::----> - INFO : new T0-Value: _RF = -1.34
ToyExclusion::----> - INFO : Skipping paramater: _gamma_
ToyExclusion::----> - INFO : new T0-Value: _alpha_ = 0.39
ToyExclusion::----> - INFO : Skipping paramater: _eta_
ToyExclusion::----> - INFO : new T0-Value: _acceptance_par_ = -0.62
Finding maximum of xe1T
Total of 6 active parameters, don't freeze parameters of interest, Minuit fits 6 param.
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest      0.1                 
  1 ERscale              Free              -5                 
  2 _PY                  Nuisance     5.8e-05                 
  3 _RF                  Nuisance    -1

Stat::pdfLikelihood::computeTheLogLikelihood - WARNING : NsFs + NbFb <= 0 
Stat::pdfLikelihood::computeTheLogLikelihood - WARNING : NsFs + NbFb <= 0 
Stat::pdfLikelihood::computeTheLogLikelihood - WARNING : NsFs + NbFb <= 0 
ML -316.602 achieved for 
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest      7.5 +- 4.7      
  1 ERscale              Free             2.7 +- 1.7      
  2 _PY                  Nuisance        -0.5 +- 0.013    
  3 _RF                  Nuisance        -1.3 +- 1.1      
  4 _gamma_              Fixed              0                 
  5 _alpha_              Nuisance       -0.66 +- 1.1      
  6 _eta_                Fixed              0                 
  7 _acceptance_par_     Nuisance          -1 +- 1.1      
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest      7.5                 
  1 ERscale              Free             2.7     

ML -302.05 achieved for 
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest      4.3 +- 3        
  1 ERscale              Free             3.4 +- 1.8      
  2 _PY                  Nuisance           1 +- 0.0038   
  3 _RF                  Nuisance       -0.86 +- 1.2      
  4 _gamma_              Fixed              0                 
  5 _alpha_              Nuisance        0.64 +- 1.7      
  6 _eta_                Fixed              0                 
  7 _acceptance_par_     Nuisance        0.66 +- 1.7      
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest      4.3                 
  1 ERscale              Free             3.4                 
  2 _PY                  Nuisance           1                 
  3 _RF                  Nuisance       -0.86                 
  4 _gamma_              Fixed              0                 
  5 _alpha_             

ML -322.889 achieved for 
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest      2.9 +- 3        
  1 ERscale              Free             4.1 +- 1.7      
  2 _PY                  Nuisance         0.5 +- 0.0023   
  3 _RF                  Nuisance       0.046 +- 1.1      
  4 _gamma_              Fixed              0                 
  5 _alpha_              Nuisance        0.68 +- 1.1      
  6 _eta_                Fixed              0                 
  7 _acceptance_par_     Nuisance       -0.31 +- 1.3      
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest      2.9                 
  1 ERscale              Free             4.1                 
  2 _PY                  Nuisance         0.5                 
  3 _RF                  Nuisance       0.046                 
  4 _gamma_              Fixed              0                 
  5 _alpha_            

ML -306.462 achieved for 
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest      5.3 +- 4.5      
  1 ERscale              Free             2.9 +- 1.5      
  2 _PY                  Nuisance         1.2 +- 0.0078   
  3 _RF                  Nuisance        0.42 +- 1.2      
  4 _gamma_              Fixed              0                 
  5 _alpha_              Nuisance       0.045 +- 1.5      
  6 _eta_                Fixed              0                 
  7 _acceptance_par_     Nuisance        0.18 +- 1.6      
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest      5.3                 
  1 ERscale              Free             2.9                 
  2 _PY                  Nuisance         1.2                 
  3 _RF                  Nuisance        0.42                 
  4 _gamma_              Fixed              0                 
  5 _alpha_            